# Install and import libraries

In [ ]:
#Install and import libraries
! pip3 install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow
!pip install transformers

In [ ]:
pip install tensorflow==2.12.0 tensorflow-probability==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 36.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-p

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load the model

In [ ]:
#Load the model

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
sample = '/content/common_voice_mr_31671592.wav'
result = pipe(sample, generate_kwargs={"language": "marathi"})
print(result["text"])

 बिहुच्या सहव्या दिवशी तरूनी हाता पायावर मेंजी कड़ुन गेतार।


In [ ]:
import os
import gdown

#Function to get transcription of audio files

def get_transcription(filename: str):

    file_path = os.path.join(folder_path, filename)

    result = pipe(file_path, generate_kwargs={"language": "marathi"})

    return f"{filename} {result['text']}"

def process_and_save_transcriptions(folder_path: str, batch_size: int, output_file_path: str):
    all_results = []

    audio_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.wav')]

    for i in range(0, min(len(audio_files), batch_size), batch_size):
        batch_files = audio_files[i:i+batch_size]

        for filename in batch_files:
            transcription = get_transcription(filename)

            all_results.append(transcription)

    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write('\n'.join(all_results))

    print(f'Processed {len(all_results)} files. Results saved to: {output_file_path}')

folder_path = '/content/drive/MyDrive/common_voice_test'
batch_size = 1816
output_file_path = '/content/drive/MyDrive/common_voice_text_results(8).txt'

process_and_save_transcriptions(folder_path, batch_size, output_file_path)

# Download the file
file_id = '/content/drive/MyDrive'  # Replace with the actual file ID from the Google Drive link
gdown.download(f'https://drive.google.com/uc?id={file_id}', output_file_path + '_downloaded.txt')

print(f'Text file downloaded to: {output_file_path}_downloaded.txt')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentiall

Processed 1816 files. Results saved to: /content/drive/MyDrive/common_voice_text_results(8).txt
Access denied with the following error:
Text file downloaded to: /content/drive/MyDrive/common_voice_text_results(8).txt_downloaded.txt



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=/content/drive/MyDrive 



In [ ]:
from google.colab import files

uploaded = files.upload()

with open('/content/trans.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()


sorted_lines = sorted(lines, key=lambda x: int(x.split('_')[-1].split('.')[0]))


with open('sorted_trans.txt', 'w', encoding='utf-8') as file:
    file.writelines(sorted_lines)

files.download('sorted_trans.txt')

KeyboardInterrupt: 

In [ ]:
from google.colab import files

uploaded = files.upload()

with open('/content/common_voice_text_results(8) (1).txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()


sorted_lines = sorted(lines, key=lambda x: int(x.split('_')[-1].split('.')[0]))

# Write the sorted content to a new file
with open('sorted_voice(8).txt', 'w', encoding='utf-8') as file:
    file.writelines(sorted_lines)

files.download('sorted_voice(8).txt')

Saving common_voice_text_results(8)(1).txt to common_voice_text_results(8)(1) (3).txt


ValueError: invalid literal for int() with base 10: '30629602 (1)'

In [ ]:
!pip install jiwer

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
import jiwer
import re

# Function to calculate Word Error Rate between two sentences
def calculate_wer(reference, hypothesis):
    try:
        return jiwer.wer(reference, hypothesis)
    except ValueError as e:
        print(f"Error calculating WER: {e}")
        return None

with open('/content/sorted_trans(1).txt', 'r', encoding='utf-8') as file1:
    trans_lines = file1.readlines()

with open('/content/sorted_common_voice(8).txt', 'r', encoding='utf-8') as file2:
    common_voice_lines = file2.readlines()

if len(trans_lines) != len(common_voice_lines):
    print("The number of sentences in the two files is different.")

# Calculate WER for each pair of sentences
min_length = min(len(trans_lines), len(common_voice_lines))
wer_results = []
for ref, hyp in zip(trans_lines[:min_length], common_voice_lines[:min_length]):
    reference = re.findall(r'\b\w+\b', ref)
    hypothesis = re.findall(r'\b\w+\b', hyp)

    max_length = max(len(reference), len(hypothesis))
    reference += [''] * (max_length - len(reference))
    hypothesis += [''] * (max_length - len(hypothesis))

    wer = calculate_wer(reference, hypothesis)
    if wer is not None:
        wer_results.append(wer)

with open('wer_results.txt', 'w', encoding='utf-8') as wer_file:
    for wer in wer_results:
        wer_file.write(f'{wer}\n')

from google.colab import files
files.download('wer_results.txt')


ModuleNotFoundError: No module named 'jiwer'